## Note: mode1-taxi, mode2-FHV, mode3-shared FHV, mode4-PT, mode5-walking

In [1]:
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

## Data

In [2]:
acs = pd.read_csv('final_acs_transportation_choice.csv')#, index_col=0)
acs.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
0,3.0,0.228957,35.391246,14.547350,7706.507979,949.324468
1,4.0,46.244797,152.049702,91.310873,7487.249289,2840.145340
2,5.0,0.349401,56.498123,9.104389,7989.174863,173.873224
3,6.0,0.263186,18.439186,4.534672,4767.467108,478.295847
4,7.0,29.894066,167.041570,44.019826,33307.536619,3139.507920


In [3]:
mode_data = pd.read_csv('final_allMode_with_wage_cleaned_update.csv', index_col=0)
mode_data = mode_data.dropna()
mode_data.head()

,12500,125000,17500,22500,225000,2500,30000,42500,62500,7500,87500,DOlocationID,PUlocationID,duration,mode,nest,price
ODpair,,,,,,,,,,,,,,,,,
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,39.695000,2,1,64.000000
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,45.216667,3,1,61.500000
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,83.000000,4,2,5.500000
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,225.933333,5,3,0.000000
3-7,3.890709,4.444350,1.693267,2.488532,3.084029,1.504915,3.552056,4.027693,5.63154,2.161294,3.521615,7,3,47.880952,1,1,43.157143


In [4]:
mode_data_2 = mode_data.copy()

In [5]:
# mode_data_2.reset_index(inplace=True)

In [6]:
mode_data_2.columns

Index(['12500', '125000', '17500', '22500', '225000', '2500', '30000', '42500',
       '62500', '7500', '87500', 'DOlocationID', 'PUlocationID', 'duration',
       'mode', 'nest', 'price'],
      dtype='object')

In [7]:
# mode_data_2['O'] = mode_data_2['ODpair'].apply(lambda x: str(x).split('-')[0]).astype('int')
# mode_data_2['D'] = mode_data_2['ODpair'].apply(lambda x: str(x).split('-')[1]).astype('int')

In [8]:
# mode_data_2.set_index('ODpair',inplace=True)

In [9]:
selected_zones = [140,141, 237, 236, 263, 262, 43, 238, 239, 143,142,12, \
                  88, 261, 13, 87, 209, 231, 45, 232, 148, 144, 211, 125, 158,\
                  249, 114, 113, 79, 4, 224, 107, 234, 90, 68, 246, 186, 164, 100,\
                  170, 137, 233, 162, 161, 230, 48, 50, 163, 229]

In [10]:
mode_data_2 = mode_data_2[(mode_data_2.PUlocationID.isin(selected_zones)) | (mode_data_2.DOlocationID.isin(selected_zones))]

In [11]:
mode_data_2.head()

,12500,125000,17500,22500,225000,2500,30000,42500,62500,7500,87500,DOlocationID,PUlocationID,duration,mode,nest,price
ODpair,,,,,,,,,,,,,,,,,
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.938590,0.360216,0.586936,4,3,39.695000,2,1,64.0
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.938590,0.360216,0.586936,4,3,45.216667,3,1,61.5
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.938590,0.360216,0.586936,4,3,83.000000,4,2,5.5
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.938590,0.360216,0.586936,4,3,225.933333,5,3,0.0
3-13,1.404978,1.604904,0.611458,0.898637,1.113677,0.543441,1.282687,1.454445,2.033612,0.780467,1.271694,13,3,54.867754,2,1,69.5


In [12]:
mode_data_2.loc[mode_data_2['mode']==1,'price'] = mode_data_2.loc[mode_data_2['mode']==1,'price']+2.5
mode_data_2.loc[mode_data_2['mode']==2,'price'] = mode_data_2.loc[mode_data_2['mode']==2,'price']+2.75
mode_data_2.loc[mode_data_2['mode']==3,'price'] = mode_data_2.loc[mode_data_2['mode']==3,'price']+0.75

In [13]:
mode_data_2.head()

,12500,125000,17500,22500,225000,2500,30000,42500,62500,7500,87500,DOlocationID,PUlocationID,duration,mode,nest,price
ODpair,,,,,,,,,,,,,,,,,
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.938590,0.360216,0.586936,4,3,39.695000,2,1,66.75
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.938590,0.360216,0.586936,4,3,45.216667,3,1,62.25
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.938590,0.360216,0.586936,4,3,83.000000,4,2,5.50
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.938590,0.360216,0.586936,4,3,225.933333,5,3,0.00
3-13,1.404978,1.604904,0.611458,0.898637,1.113677,0.543441,1.282687,1.454445,2.033612,0.780467,1.271694,13,3,54.867754,2,1,72.25


In [14]:
mode_data_2.shape

(90345, 17)

## Modeling

### Nested Logit Model

In [15]:
def utility(mode, wage, Lambda, dataset):
    '''
    Get the utility for mode j under specific OD pair
    Lambda: parameter that trade-off different transportation mode
    '''
    subset = dataset[dataset['mode'] == mode]
    vj = Lambda * (float(int(wage)/124800) * float(subset['duration']) + float(subset['price'])) #525600: convert wage scale 'year' to 'minitues'
    return -vj #assign negative utility?


def InclusiveValue(Nk, Tk, wage, dictVj, Lambda, dataset):
    '''
    Get the inclusive value for nest K
    Nk:nest k
    T: the dict that contains Tao(dissmilarity parameter) for each Nest. for example, Tk meeas Tao for nest K
    '''
    subsetNk = dataset[dataset['nest'] == Nk]
    modes = list(subsetNk['mode'].unique()) #what modes contained in this nest k
    sumIV = 0
    for j in modes:   
        vj = utility(j, wage, Lambda, subsetNk)
        dictVj[j] = vj
        sumIV += np.exp((1/Tk)*vj)  
    IVk = np.log(sumIV)
    return dictVj, IVk
    

def denoSum(T, nestList, wage, Lambda, dataset):
    '''
    Calculate the denomenator for P(y=Nk)
    T: the dict that contains Tk for each Nest, in our case T={1:T1, 2:T2, 3:T3}; T1, T2, T3 defined by us 
    TotK: the total number of nest this model has, in our case TotK = 3
    '''
    deno = 0
    dictIVk = {}
    dictVj = {}
    for Nk in nestList: #k is the k Nest, in our case k=1,2,3
        Tk = T[Nk] #get the tao for nest k 
        dictVj, IVk = InclusiveValue(Nk, Tk, wage, dictVj, Lambda, dataset)
        denok = np.exp(Tk*IVk)
        deno += denok
        dictIVk[Nk] = IVk
    return dictVj, dictIVk, deno


def probability (j, Nk, T, dictVj, dictIVk, deno):  
    '''
    Calculate the probability for the specific mode j and Nest Nk
    '''
    Tk = T[Nk]
    IVk = dictIVk[Nk]
    vj = dictVj[j]
    pjk = np.exp((1/Tk)*vj)/np.exp(IVk) #pjk: P(y=j, y belong to Nk)
    pk = np.exp(Tk*IVk)/deno #pk: P(y belong to Nk)
    pj = pjk*pk #pj: P(y=j)
    return pj

### Apply to our case

In [16]:
def apply_model(T, wageList, Lambda, dataAll):
    '''
    apply to our case
    '''
    import timeit
    start = timeit.default_timer()

    ODpair_list = list(dataAll.index.unique())
    p = []
    loss_function_deno = []
    for od in ODpair_list: #indentify OD pair  
        dataOD = dataAll[dataAll.index==od] 
        modeList = list(dataOD['mode'])
        nestList = list(dataOD['nest'])
        
        pop_OD = [] #store the results under each OD pair
        loss_function_deno_OD = [] #store the denominator of the loss function under each OD pair
        for wage in wageList:            
            dictVj, dictIVk, deno = denoSum(T, set(nestList), wage, Lambda, dataOD)
            pop_mode = [] #store the population results under each OD pair and each wage
            loss_function_deno_mode = [] #store the deno of loss function under each OD pair and each wage
            for i in range(1,6):
                if i in modeList: #not all modes appear in every OD pair
                    prob = probability(i, nestList[modeList.index(i)], T, dictVj, dictIVk, deno) #probability under OD pari and mode i
                    if np.isnan(prob) == True: #if predicted probability is nan, replace it as 0, means no people choose
                        prob = 0
                    
                    pop = dataOD[wage].mean() * prob
                    loss_function_denoi = dataOD[wage].mean() * (prob - prob**2)               
                    pop_mode.append(pop)
                    loss_function_deno_mode.append(loss_function_denoi)
                else: 
                    pop_mode.append(0) #the probability of mode which not in the modeList is also 0
                    loss_function_deno_mode.append(0)
                    #pop_mode.append(1)
            pop_OD.append(pop_mode) #the shape of pop_OD is len(wageList) * 5 
            loss_function_deno_OD.append(loss_function_deno_mode)
        pop_OD_sum = [sum(x) for x in zip(*pop_OD)] #sum the population of each wage 
        loss_function_deno_sum = [sum(x) for x in zip(*loss_function_deno_OD)]
        p.append(pop_OD_sum)
        loss_function_deno.append(loss_function_deno_sum)

    df = pd.DataFrame(p, columns=['P(mode1)', 'P(mode2)', 'P(mode3)', 'P(mode4)', 'P(mode5)']
                      , index=ODpair_list)
    df_LF_deno = pd.DataFrame(loss_function_deno, columns=['mode1', 'mode2', 'mode3', 'mode4', 'mode5']
                               , index=ODpair_list)
    
    stop = timeit.default_timer()
    timeslot = stop - start
    return df, df_LF_deno, timeslot

def compare_with_ground_truth(predictdf, loss_function_deno, truedf):
    '''
    compare our predicted transportation choice with ground truth
    
    The header of the datafrme after merge (named 'data_compare') should be like:
    taxi_zone | P(mode1)_x | P(mode2)_x | P(mode3)_x | P(mode4)_x | P(mode5)_x | P(mode1)_y | P(mode2)_y | P(mode3)_y | P(mode4)_y | P(mode5)_y
    '''
    import numpy as np
    # makesure predictdf and truedf have the same formats
    for col in predictdf.columns:
        predictdf[col] = predictdf[col].astype(float) 
    predictdf = predictdf.fillna(0)
    predictdf = predictdf.replace([np.inf, -np.inf], np.nan)
    predictdf = predictdf.dropna()
    predictdf['taxi_zone'] = predictdf.index.map(lambda x: x.split('-')[0]) #get origin taxi zone from each OD pair
    predictdf = predictdf.groupby('taxi_zone').sum().reset_index() #group the popuation by taxi zone
    predictdf['taxi_zone'] = predictdf['taxi_zone'].astype(int)
    
    #do same thing as predictdf to loss_function_deno dataframe
    for col in loss_function_deno.columns:
        loss_function_deno[col] = loss_function_deno[col].astype(float) 
    loss_function_deno = loss_function_deno.fillna(0)
    loss_function_deno = loss_function_deno.replace([np.inf, -np.inf], np.nan)
    loss_function_deno = loss_function_deno.dropna()
    loss_function_deno['taxi_zone'] = loss_function_deno.index.map(lambda x: x.split('-')[0]) #get origin taxi zone from each OD pair
    loss_function_deno = loss_function_deno.groupby('taxi_zone').sum().reset_index() #group the popuation by taxi zone
    loss_function_deno['taxi_zone'] = loss_function_deno['taxi_zone'].astype(int)   
    
    truedf['taxi_zone'] = truedf['taxi_zone'].astype(int)
    
    data_compare = pd.merge(predictdf, truedf, left_on='taxi_zone', right_on = 'taxi_zone')
#     data_compare = pd.merge(predictdf, truedf, left_on='taxi_zone', right_on = 'taxi_zone', how='left')
    data_compare = data_compare.dropna()  #some zone in predictdf are not exist in acs!!!!?????
    data_compare_zone = data_compare['taxi_zone'].unique()
    loss_function_deno = loss_function_deno[loss_function_deno['taxi_zone'].isin(data_compare_zone)]
    
    loss = 0
    rloss = 0
    for i in range(1,6):
        #define the loss function
        lossi = sum((data_compare[data_compare.columns[i]] - data_compare[data_compare.columns[i+5]])**2/loss_function_deno[loss_function_deno.columns[i]])
        rlossi = np.sqrt(lossi)
    loss += lossi
    rloss += rlossi
    return data_compare, loss_function_deno, loss, rloss

In [59]:
#Run the model for the scnario 2
T1 = 0.1 #should set grid search for T1 
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} #Tao for each nest
Lambda = 0.22 #should set grid search for Lambda 
wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']

predict_choice_2, loss_function_denodf_2, timeslot_2 = apply_model(T, wagelist, Lambda, mode_data_2)
combine_2, loss_deno_2, loss_2, rloss_2 = compare_with_ground_truth(predict_choice_2, loss_function_denodf_2, acs)

KeyboardInterrupt: 

In [58]:
54/239

0.22594142259414227

In [45]:
zeros = list(predict_choice_2[predict_choice_2['P(mode1)'] == 0 ].index)

In [50]:
zeros = [int(element.split('-')[0]) for element in zeros]

In [52]:
zeros = list(set(zeros))

In [57]:
loss_deno_2[loss_deno_2.taxi_zone.isin(zeros)].shape

(239, 6)

In [53]:
zeros

[3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 106,
 107,
 108,
 109,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193

In [41]:
predict_choice_2.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-4,0.0,0.0,0.0,5.619661,0.380339
3-13,0.0,0.0,0.0,12.429913,0.570087
3-43,0.0,0.0,0.0,8.522003,0.477997
3-45,0.0,0.0,0.0,9.574366,0.425634
3-48,0.0,0.0,0.0,61.721259,3.278741


In [25]:
predict_choice_2.to_csv('results_scenario2`.csv')